In [1]:
import os
import numpy as np
import pandas as pd
import time
import scipy.stats as stats
from scipy.special import factorial
import csv
import matplotlib.pyplot as plt
import statsmodels.stats.multitest as sm
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
import math

# Import data

In [2]:
imulabel = pd.read_csv('../data/immune_related_genes.csv')['Gene Name'].values

df1 = pd.read_csv('./ocRMA/exp.mRNA.AgilentG4502A_07_2.txt',sep = "\t", index_col=0,header = 0)
df2 = pd.read_csv('./ocRMA/exp.mRNA.HT_HG-U133A.txt',sep = "\t", index_col=0,header = 0)
df3 = pd.read_csv('./ocRMA/exp.mRNA.GSE6008.RMA.txt',sep = "\t", index_col=0,header = 0)
df4 = pd.read_csv('./ocRMA/exp.mRNA.GSE18520.RMA.txt',sep = "\t", index_col=0,header = 0)
df5 = pd.read_csv('./ocRMA/exp.mRNA.GSE40595.RMA.txt',sep = "\t", index_col=0,header = 0)
df6 = pd.read_csv('./ocRMA/mRNA.validation.txt',sep = "\t", index_col=0,header = 0)
df6 = df6.drop(df6.columns[range(8,16)],axis=1)

# GSE18520 + TCGA_U133a
df7 = pd.read_csv('./ocRMA/combat.ocdata.txt',sep="\t",index_col=0,header=0)

imuidx = pd.Index(imulabel)
comidx = df1.index.join(
    df2.index.join(
        df3.index.join(
            df4.index.join(
                df5.index.join(
                    df6.index.join(imuidx,how='inner'),how='inner')
            ,how='inner')
        ,how='inner')
    ,how='inner')
,how='inner')

comidx = df7.index.join(comidx,how='inner')

df1 = df1.loc[comidx]
df2 = df2.loc[comidx]
df3 = df3.loc[comidx]
df4 = df4.loc[comidx]
df5 = df5.loc[comidx]
df6 = df6.loc[comidx]
df7 = df7.loc[comidx]

print(df1.shape)
print(df2.shape)
print (df3.shape)
print (df4.shape)
print (df5.shape)
print (df6.shape)
print (df7.shape)

(586, 37)
(586, 593)
(586, 45)
(586, 63)
(586, 38)
(586, 16)
(586, 656)


In [3]:
lab1 = np.genfromtxt('./oclabel/AgilentG4502A_07_2.label.txt',dtype="U16,u1")['f1']
lab2 = np.genfromtxt('./oclabel/HT_HG-U133A.label.txt',dtype="U16,u1")['f1']
lab3 = np.genfromtxt('./oclabel/GSE6008.label.txt',dtype="U16,U16,u1")['f2']
lab4 = np.genfromtxt('./oclabel/GSE18520.label.txt',dtype="U16,U16,U16,u1,u1")['f4']
lab5 = np.genfromtxt('./oclabel/GSE40595.label.txt',dtype="U16,U16,U16,U16,U16,U16,U16,u1")['f7']
lab6 = np.genfromtxt('./oclabel/mRNA.label.txt',dtype="U16,u1")['f1']
lab7 = np.genfromtxt('./oclabel/combat.label.txt',dtype="U16,u1")['f1']
lab6 = np.delete(lab6,range(8,16))

# Set training and Testing

In [4]:
# trainname = "RMA_testonMAS"
# x_train = np.concatenate((df4.to_numpy(),df2.to_numpy()),axis=1).T
# y_train = np.concatenate((lab4,lab2))
x_train = df7.to_numpy().T
y_train = lab7

x_test1 = df3.to_numpy().T
x_test2 = df5.to_numpy().T
x_test3 = df1.to_numpy().T
x_test4 = df6.to_numpy().T
y_test1 = lab3
y_test2 = lab5
y_test3 = lab1
y_test4 = lab6

length = len(x_train[0,:])
n_sm = len(x_train[:,0])
print ("Shape of training data: ",x_train.shape)
print ("length of data = ", length)
print ("number of samples = ", n_sm)

Shape of training data:  (656, 586)
length of data =  586
number of samples =  656


# Ind_T-test

In [5]:
def select(data, index):
    index_num = len(index)
    select = []
    for i in range(0,index_num):
        select.append(np.array(data[:,index[i]]))
    select = np.stack(select)
    select = select.T
#     print "shape of select matrix: ", select.shape
    return select

In [6]:
con_index = np.where(y_train==0)
con_sample = x_train[con_index]
case_index = np.where(y_train==1)
case_sample = x_train[case_index]
print ("shape of control", con_sample.shape)
print ("shape of case", case_sample.shape)

shape of control (18, 586)
shape of case (638, 586)


In [7]:
t_stat,pvalue = stats.ttest_ind(con_sample, case_sample, axis = 0, equal_var=True, nan_policy='raise')
print (np.min(pvalue))
rejected, result_fdr = sm.fdrcorrection(pvalue, method='indep', is_sorted=False)
# result_fdr = pvalue
print (np.min(result_fdr))

2.26438692667406e-69
1.3269307390309992e-66


In [8]:
# fdr_thresholds = np.arange(0.01,0.06,0.01)
fdr_thresholds = [1e-4,1e-10,1e-12,1e-16,1e-20,1e-28,1e-36]
for fdr_threshold in fdr_thresholds:
    index, = np.where(result_fdr < fdr_threshold)
    print ("fdr_threshold = ", fdr_threshold)
    print (len(index))

fdr_threshold =  0.0001
109
fdr_threshold =  1e-10
28
fdr_threshold =  1e-12
24
fdr_threshold =  1e-16
13
fdr_threshold =  1e-20
8
fdr_threshold =  1e-28
5
fdr_threshold =  1e-36
2


# Lasso

In [9]:
def train_Lasso(data,label,plot=False):
    # Compute paths
    print("Computing regularization path using the coordinate descent lasso...")
    model = linear_model.LassoCV(max_iter=1000,cv=10,normalize=True ,random_state=2).fit(data, label)

    # This is to avoid division by zero while doing np.log10
    EPSILON = 1e-5

    # Display results
    if plot:
        m_log_alphas = -np.log10(model.alphas_ + EPSILON)

        plt.figure()
        ymin, ymax = 0, 0.6
        plt.plot(m_log_alphas, model.mse_path_, ':')
        plt.plot(m_log_alphas, model.mse_path_.mean(axis=-1), 'k',
                 label='Average across the folds', linewidth=2)
        plt.axvline(-np.log10(model.alpha_ + EPSILON), linestyle='--', color='k',
                    label='alpha: CV estimate')

        plt.legend()

        plt.xlabel('-log(alpha)')
        plt.ylabel('Mean square error')
        plt.title('Mean square error on each fold: coordinate descent '
                  '(train time: %.2fs)' % t_lasso_cv)
        plt.axis('tight')
        plt.ylim(ymin, ymax)

    num_coef = len(np.where(model.coef_)[0])
    print ("num_lasso_gene = ", num_coef)
    return model, num_coef

In [10]:
def test_Lasso(data, label, clf):
    predt = clf.predict(data)
    auc = metrics.roc_auc_score(label, predt)
    print ("Lasso ROC AUC: ", auc)
#     plot_roc(y_true,y_scores)
    return auc

In [11]:
def plot_roc(labels, predict_prob):
    false_positive_rate,true_positive_rate,thresholds=\
    metrics.roc_curve(labels, predict_prob)
    roc_auc = metrics.auc(false_positive_rate, true_positive_rate)
    plt.title('ROC')
    plt.plot(false_positive_rate, true_positive_rate,'b',\
             label = 'AUC = %0.4f'% roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0,1],[0,1],'r--')
    plt.ylabel('TPR')
    plt.xlabel('FPR')
    plt.show()

# Set field name of the result

In [12]:
fields = ['threshold','num_gene','num_lassogene','lasso_test']
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]
for name in names:
    fields.append(name)

print (fields)

['threshold', 'num_gene', 'num_lassogene', 'lasso_test', 'Nearest Neighbors', 'Linear SVM', 'RBF SVM', 'Gaussian Process', 'Decision Tree', 'Random Forest', 'Neural Net', 'AdaBoost', 'Naive Bayes', 'QDA']


# Other Classifier

In [13]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [14]:
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025,random_state=2),
    SVC(gamma=2, C=1,random_state=2),
    GaussianProcessClassifier(1.0 * RBF(1.0),random_state=2),
    DecisionTreeClassifier(random_state=2),
    RandomForestClassifier(random_state=2),
    MLPClassifier(hidden_layer_sizes=(20,20),alpha=0.5, max_iter=1000,learning_rate='adaptive', random_state=2),
    AdaBoostClassifier(random_state=2),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

# Fold Change

In [15]:
def foldchange(x,y,fc):
    case = y
    control  = 1-y
    casemean = np.sum(x.T * case, axis = 1) / np.sum(case)
    controlmean = np.sum(x.T * control, axis = 1) / np.sum(control)
    index, = np.where(abs(casemean - controlmean) > math.log(fc,2))
#     index, = np.where(abs(casemean - controlmean) > fc)
    return index

In [16]:
fc_index = foldchange(x_train,y_train,2)
fc_index.shape

(63,)

In [25]:
fdr_index, = np.where(result_fdr < 0.0001)
print("fdr_index = ", len(fdr_index))
fc_index = foldchange(x_train,y_train,2)
print("fc index = ", len(fc_index))
index = np.intersect1d(fdr_index,fc_index)
print("intersect index = ", len(index))

fdr_index =  109
fc index =  63
intersect index =  47


## Training and Testing

In [17]:
def testmodel(clf,x_test,y_test,index):
    testrsl = []
    testrsl.append('fdr<'+'%.0e' % threshold)
    testrsl.append(len(index))
    
    etc_test = x_test[:,index]
    lassoauc = test_Lasso(etc_test,y_test,clf)
    testrsl.append(num_lasso_gene)
    testrsl.append('%.3f' % lassoauc)
    
    for name, clf_ in zip(names, classifiers):
        clf_.fit(etc_train,y_train)
        predt = clf_.predict(etc_test)
        auc = metrics.roc_auc_score(y_test, predt)
        print (name, "Testing ROC AUC: ", auc)
        testrsl.append('%.3f' % auc)
        
    return testrsl

In [18]:
# from imblearn.over_sampling import RandomOverSampler
# ros = RandomOverSampler(random_state=0)
# x_resampled, y_resampled = ros.fit_resample(x_train, y_train)

# fdr_thresholds = [1e-4,1e-10,1e-12,1e-16,1e-20,1e-28,1e-36]
# fc = 2

# trainrsl =[]
# test1rsl = []
# test2rsl = []
# test3rsl = []
# test4rsl = []
    
# for threshold in fdr_thresholds:
    
#     fdr_index, = np.where(result_fdr < threshold)
#     fc_index = foldchange(x_train,y_train,fc)
#     index = np.intersect1d(fdr_index,fc_index)
     
#     if index.size == 0:
#         print("for threshold={:.0e}, fc={:.1f} intersection empty".format(threshold,fc))
#         continue
    
# #     etc_train = x_resampled[:,index]
#     etc_train = x_train[:,index]
#     clf, num_lasso_gene = train_Lasso(etc_train,y_train,plot=False)
    
#     trainrsl.append(testmodel(clf,x_train,y_train,index))
#     test1rsl.append(testmodel(clf,x_test1,y_test1,index))
#     test2rsl.append(testmodel(clf,x_test2,y_test2,index))
#     test3rsl.append(testmodel(clf,x_test3,y_test3,index))
#     test4rsl.append(testmodel(clf,x_test4,y_test4,index))

In [20]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
x_resampled, y_resampled = ros.fit_resample(x_train, y_train)

genenum = [7]

trainrsl = []
test2rsl = []
test3rsl = []
test4rsl = []
test5rsl = []
test6rsl = []
test7rsl = []
test8rsl = []
test9rsl = []
    
for n in genenum:
    threshold = np.sort(result_fdr)[n]
    fdr_index, = np.where(result_fdr < threshold)
    index = fdr_index
    
#     fdr_index, = np.where(result_fdr < threshold)
#     fc_index = foldchange(x_train,y_train,fc)
#     index = np.intersect1d(fdr_index,fc_index)
     
    if index.size == 0:
        print("for threshold={:.0e}, fc={:.1f} intersection empty".format(threshold,fc))
        continue
    
#     etc_train = x_resampled[:,index]
    etc_train = x_train[:,index]
    clf, num_lasso_gene = train_Lasso(etc_train,y_train,plot=False)
    
    trainrsl.append(testmodel(clf,x_train,y_train,index))
    test1rsl.append(testmodel(clf,x_test1,y_test1,index))
    test2rsl.append(testmodel(clf,x_test2,y_test2,index))
    test3rsl.append(testmodel(clf,x_test3,y_test3,index))
    test4rsl.append(testmodel(clf,x_test4,y_test4,index))

Computing regularization path using the coordinate descent lasso...
num_lasso_gene =  7
Lasso ROC AUC:  0.9995646116335771
Nearest Neighbors Testing ROC AUC:  0.9714385231626611
Linear SVM Testing ROC AUC:  0.9714385231626611
RBF SVM Testing ROC AUC:  1.0
Gaussian Process Testing ROC AUC:  0.9714385231626611
Decision Tree Testing ROC AUC:  1.0
Random Forest Testing ROC AUC:  1.0
Neural Net Testing ROC AUC:  0.9706548241030999
AdaBoost Testing ROC AUC:  1.0
Naive Bayes Testing ROC AUC:  0.9636015325670497
QDA Testing ROC AUC:  0.9675200278648554
Lasso ROC AUC:  0.9878048780487805
Nearest Neighbors Testing ROC AUC:  0.7896341463414633
Linear SVM Testing ROC AUC:  0.9512195121951219
RBF SVM Testing ROC AUC:  0.5
Gaussian Process Testing ROC AUC:  0.9146341463414633
Decision Tree Testing ROC AUC:  0.5609756097560976
Random Forest Testing ROC AUC:  0.9878048780487805
Neural Net Testing ROC AUC:  0.5
AdaBoost Testing ROC AUC:  0.5609756097560976
Naive Bayes Testing ROC AUC:  0.53658536585365

# Write to csv

In [21]:
import csv
from datetime import datetime

dateTimeObj = datetime.now()
timestr = dateTimeObj.strftime('%m%dT%H')
# # 属性名称
# fields = ['Name', 'Goals', 'Assists', 'Shots']

# # csv文件中每一行的数据，一行为一个列表
# rows = [ ['Emily', '12', '18', '112'],
# ['Katie', '8', '24', '96'],
# ['John', '16', '9', '101'],
# ['Mike', '3', '14', '82']]

# filename = "./Result/Sepsis_Results_value_Adult1+3_"+timestr+".csv"
filename = "./Result/oc_value_n=7_combat.csv"

# 将数据写入到csv文件中
with open(filename, 'w+') as csvfile:
    # 创建一个csv writer对象
    csvwriter = csv.writer(csvfile)
    # 写入属性名称
    csvwriter.writerow(fields)
    # 写入数据
    csvwriter.writerows([["train"]])
    csvwriter.writerows(trainrsl)
    csvwriter.writerows([["test1GEO"]])
    csvwriter.writerows(test1rsl)
    csvwriter.writerows([["test2GEO"]])
    csvwriter.writerows(test2rsl)
    csvwriter.writerows([["test3TCGA"]])
    csvwriter.writerows(test3rsl)
    csvwriter.writerows([["Validation"]])
    csvwriter.writerows(test4rsl)
